# Imports

In [28]:
import time
import serial
import sqlite3
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from scipy.signal import argrelextrema

# BiometricSignal Class

In [29]:
class BiometricSignal: 
    
    captured_signal_csv = "./assets/subject_raw_ecg.csv"

    def __init__(self):
#         self.capture_signal = capture_signal()
        self.filtered_signal = self.filter_captured_signal()
        self.r_peaks, self.amended_signal  = self.find_r_peaks(self.filtered_signal)
        
        

    def capture_signal(self):
        serial_data = serial.Serial("/dev/ttyACM1", 9600)
        serial_list = []
        capture_time = 5
        t_end = time.time() + capture_time

        time.sleep(2)

        while time.time() < t_end:
            while serial_data.inWaiting() == 0:
                pass
            temp_string = serial_data.readline()
            serial_string = (
                str(temp_string)
                .replace("b", "")
                .replace("'", "")
                .replace("\\r", "")
                .replace("\\n", "")
            )
            if len(serial_string) == 3:
                serial_list.append(int(serial_string))
        return serial_list


    def output_signal_to_csv(self, serial_list):
        with open(BiometricSignal.captured_signal_csv, "w") as ecg_file:
            ecg_file.write("voltage,\n")
            for item in serial_list:
                ecg_file.write("%s,\n" % item)
            ecg_file.write("0,")


    def filter_captured_signal(self):
        numerator, denominator = signal.butter(4, 0.25, analog=False)
        data = pd.read_csv(BiometricSignal.captured_signal_csv)
        sig = data["voltage"]
        filtered_signal = signal.filtfilt(numerator, denominator, sig)
        return filtered_signal
    
    
    def find_r_peaks(self, filtered_signal):
        
        threshold = 400
        start = 400
        end = -100
        
        amended_signal = filtered_signal[start:end]
        no_of_rows = amended_signal.shape[0]
        line_numbers = []
        theVoltage = []


        for i in range(0, no_of_rows):
            if amended_signal[i] > threshold:
                theVoltage.append(amended_signal[i])
            else:
                theVoltage.append(0)
            line_numbers.append(i)    


        ecg_plot = np.concatenate((theVoltage, line_numbers))

        r_peaks = argrelextrema(ecg_plot, np.greater, order=5)
        
        return r_peaks, amended_signal



        


#         c.execute("SELECT featuresHigherY FROM templates WHERE name IS 'Sam'")
#         test_final=(c.fetchone())

#         list3 = json.loads(test_final[0])
#         print(list3)
#         conn.commit()
#         conn.close()

# Segment Class

In [30]:
class Segment:

    
    
    def __init__(self, bio_signal):
        self.bio_signal = bio_signal
    
    
#         print(bio_signal.r_peaks)

    # ----------------------------------------------------
    # --------------Combining Segments -------------------
    # ----------------------------------------------------
    def combining_segments(self):
        combined_seg_does_not_exist = True
        smallest_seg = None
        
#         fig1, ax1 = plt.subplots(figsize=(10,5)) # code for demo purposes only
#         fig2, ax2 = plt.subplots(figsize=(10,5)) # code for demo purposes only
        
        for i in range (0, 5):
            segment_start = self.bio_signal.r_peaks[0][i]
            segment_end = self.bio_signal.r_peaks[0][i+1]

            extracted_segment = self.bio_signal.amended_signal[segment_start:segment_end]
            if smallest_seg == None:
                smallest_seg = len(extracted_segment)

            elif (len(extracted_segment) < smallest_seg):
                smallest_seg = len(extracted_segment)
            
#             ax1.plot(extracted_segment) # code for demo purposes only

            if combined_seg_does_not_exist:
                combined_seg = np.zeros(len(extracted_segment) + 100)
                combined_seg_does_not_exist = False
            for j in range(0,len(extracted_segment)):
                combined_seg[j] =  combined_seg[j] + extracted_segment[j]
            
#             ax2.plot(combined_seg[0:-102]) # code for demo purposes only
            
#         fig1.savefig("extracted_segment.png", dpi=150, quality=100, bbox_inches='tight')  # code for demo purposes only
#         fig2.savefig("combined_seg.png", dpi=150, quality=100, bbox_inches='tight')  # code for demo purposes only
        combined_seg = np.trim_zeros(combined_seg)
        return combined_seg[0:smallest_seg]
                

    def get_mean_of_segments(self, combined_seg):
        mean_segment = np.array([])

        for k in range (0, len(combined_seg)):
            mean_segment = np.append (mean_segment, combined_seg[k] / 5)
            
#         print(mean_segment)
        return mean_segment
        

# Features Class

In [56]:
class Features:
    
    def __init__(self, mean_segment):
        self.mean_segment = mean_segment
        
    def find_features(self):
        
        features_lower = argrelextrema(self.mean_segment, np.less, order=5)
        features_higher = argrelextrema(self.mean_segment, np.greater, order=5)
        
#         plt.figure(num=None, figsize=(10, 5), dpi=80, facecolor="w", edgecolor="k")
#         plt.axis('off')
#         plt.plot(self.mean_segment, color="#000000", linewidth=1)

        features_lower = (features_lower[0],self.mean_segment[features_lower[0]]) # removed .tolist()    
        features_higher = (features_higher[0],self.mean_segment[features_higher[0]])
        
        return features_lower, features_higher

#         plt.scatter(features_higher[0],self.mean_segment[features_higher[0]],linewidth=0.3, s=250, c='r')
#         plt.scatter(features_lower[0],self.mean_segment[features_lower[0]],linewidth=0.3, s=250, c='b')

# Templates Class

In [150]:
class Templates:
    
    def __init__(self):
        self.conn = sqlite3.connect('templatesDB.db')
        self.c = self.conn.cursor()
        self.create_table()
    
    def create_table(self):
        self.c.execute("""CREATE TABLE IF NOT EXISTS templates 
                                            ( name text,
                                                featuresLowerX text,
                                                featuresLowerY text,
                                                featuresHigherX text,
                                                featuresHigherY text)""")
        self.conn.commit()
        
    def add_entry_to_database(self, features_lower, features_higher, theName):
            
        if self.does_user_exist(theName) != True:
            self.thefeaturesLowerX = json.dumps(features_lower[0].tolist())
            self.thefeaturesLowerY = json.dumps(features_lower[1].tolist())
            self.thefeaturesHigherX = json.dumps(features_higher[0].tolist())
            self.thefeaturesHigherY = json.dumps(features_higher[1].tolist())

            self.c.execute("INSERT INTO templates VALUES (:name, :featuresLowerX, :featuresLowerY, :featuresHigherX, :featuresHigherY)",
                                                {'name': theName,
                                                 'featuresLowerX': self.thefeaturesLowerX,
                                                 'featuresLowerY': self.thefeaturesLowerY,
                                                 'featuresHigherX': self.thefeaturesHigherX,
                                                 'featuresHigherY': self.thefeaturesHigherY})

        self.conn.commit()

    def query_db(self, theName):
        self.c.execute("SELECT * FROM templates WHERE name IS (:name)", {'name': theName})
        query_response=(self.c.fetchall()) # change this on fetchone        
        return query_response
    
    def test_function(self):
        print("Hello, this is a mother fucking test")
    
    def does_user_exist(self, theName):
        if self.query_db(theName) == []:
            return False
        else:
            return True
        
    def get_template(self, theName):
        self.does_user_exist(theName)
        query_response = self.query_db(theName)
        featuresLowerX = json.loads(query_response[0][1])
        featuresLowerY = json.loads(query_response[0][2])
        featuresHigherY = json.loads(query_response[0][3])
        featuresHigherY = json.loads(query_response[0][4])
        return featuresLowerX, featuresLowerY, featuresHigherX, featuresHigherY
        
        


In [157]:
# ----------------------------------------------------------------------------------------------
#                                       CODE FOR TESTING
# ----------------------------------------------------------------------------------------------
testSignal = BiometricSignal()
testSegment = Segment(testSignal)
combined_seg = testSegment.combining_segments()
# print(smallest_seg)
# print(len(combined_seg[0:124]))
testMeanSegment = testSegment.get_mean_of_segments(combined_seg)
testFeatures = Features(testMeanSegment)
# testFindFeatures = testFeatures.find_features()
features_lower, features_higher = testFeatures.find_features()

testTemplates = Templates()
testTemplates.create_table()

theName = 'Sam'
testTemplates.add_entry_to_database(features_lower, features_higher, theName)
# query_response = testTemplates.query_db(theName)
featuresLowerX, featuresLowerY, featuresHigherX, featuresHigherY = testTemplates.get_template(theName)


# featuresHigherY = json.loads(query_response[0][1])


featuresLowerX, featuresLowerY, featuresHigherX, featuresHigherY = testTemplates.get_template(theName)
# list3 = json.loads(query_response[0][1])
print(type(featuresHigherY))
print(featuresHigherY)
# testTemplates.query_db(theName)
# testTemplates.does_user_exist(theName)
# testTemplates.test_function()
# type(testTemplates)


<class 'list'>
[323.29507837770916, 369.7063509557241, 318.8277261330069, 336.6725340035738, 318.130779591436]


In [ ]:
# plt.figure(num=None, figsize=(10, 5), dpi=80, facecolor="w", edgecolor="k")
# plt.plot(filtered_signal, color="#000000", linewidth=1)
# plt.axis('off')
# plt.savefig("./assets/filter_signal.png", dpi=150, quality=100, bbox_inches='tight')
# plt.tight_layout()
# plt.show()